# Защита персональных данных клиентов

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Постановка-задачи" data-toc-modified-id="Постановка-задачи-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Постановка задачи</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Выводы-и-рекомендации" data-toc-modified-id="Выводы-и-рекомендации-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Выводы и рекомендации</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Загрузка данных

In [1]:
!pip install brewer2mpl -q

In [2]:
!pip install cufflinks plotly -q

In [3]:
!pip install chart-studio -q

In [4]:
import chart_studio.plotly as py
import cufflinks
cufflinks.go_offline(connected=True)
import pandas as pd
import seaborn as sns
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import warnings; warnings.filterwarnings(action='once')

In [5]:
data = pd.read_csv('/datasets/insurance.csv')
display(data.head(5))

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [6]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


Таблица содержит 5000 строк с данными клиентов, представленных в 5столбцах. Типы данных колонок - `float` и `int`. Явных пропусков нет.

## Предобработка данных

In [7]:
print('Количество явных пропусков:', data.isna().sum())

Количество явных пропусков: Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64


In [8]:
columns = data.columns
for column in columns:
    print('Количество нулей в графе', column, ':', data[data[column] == 0][column].count())

Количество нулей в графе Пол : 2505
Количество нулей в графе Возраст : 0
Количество нулей в графе Зарплата : 0
Количество нулей в графе Члены семьи : 1513
Количество нулей в графе Страховые выплаты : 4436


Значение 0 в столбце пола - категориальное; нулевых возрастов и зарплат нет, что нормально, в колонке членов семьм есть нулевые значения, что вероятно означает отсутствие ближайших родственников, нули в столбце страховых выплат - также категориальные значения, означающие отсутствие таких выплат. 

In [9]:
print('Количество явных дубликатов:', data.duplicated().sum())

Количество явных дубликатов: 153


*Исследование данных, поиск странностей:*

In [10]:
data['Пол'].iplot(kind='hist', xTitle='Пол',
                  yTitle='Количество', title='Распределение полов в датасете')

Оба пола представлены примерно поровну

In [11]:
data['Возраст'].iplot(kind='hist', xTitle='Возраст',
                  yTitle='Количество', title='Распределение возрастов')

При относительно небольшом числе возрастов наблюдаем пуассоновское распределение.

In [12]:
data['Зарплата'].iplot(kind='hist', xTitle='Зарплата',
                  yTitle='Количество', title='Распределение зарплат')

Вариативность наблюдаемых зарплат большая, данные распределены нормально. Есть единичные небольшие выбросы с обеих сторон, выбросы слева могут быть ошибками при заполнении данных.

In [13]:
data['Члены семьи'].iplot(kind='hist', xTitle='Члены семьи',
                  yTitle='Количество', title='Распределение количества членов семьи клиента')

Крайне редко встречается количество членов семьи болье 4, соответственно имеем пуассоновское распределение с сильным смещением влево. Выбросов нет.

Так как взаимосвязь данных проанализировать необходимо, но матрица корелляции тут не подходит, было решено использовать анализ дисперсии `ANOVA`. ANOVA используется для сравнения средних значений количественного признака в зависимости от значений качественного признака.

Используем библиотеку `statsmodels` для создания модели ANOVA. Таблица ANOVA показывает, есть ли статистически значимые различия между количественными и качественными признаками. Формула Price ~ C(Region) означает, что мы хотим анализировать зависимость цены от региона, а C(Region) указывает, что Region является качественным признаком. Метод fit() применяет модель к данным, а метод anova_lm() вычисляет таблицу ANOVA.

Таблица ANOVA показывает, что есть статистически значимые различия в средней цене продажи автомобилей между регионами (p-value = 0.021978). Это означает, что средняя цена продажи автомобилей зависит от региона, и мы можем использовать эту информацию для принятия решений в бизнесе (например, для установления различных цен в разных регионах).

In [14]:
print(data.columns)

Index(['Пол', 'Возраст', 'Зарплата', 'Члены семьи', 'Страховые выплаты'], dtype='object')


In [15]:
data.rename(columns={'Члены семьи': 'Члены_семьи', 'Страховые выплаты': 'Страховые_выплаты'}, inplace=True)

In [16]:
# Вычисление корреляции Спирмена и p-значения
corr_matrix = data.corr(method='spearman')
p_values = data.corr(method=lambda x, y: spearmanr(x, y)[1])
display('Матрица корелляции Спирмена:', corr_matrix)
display('P-values:', p_values)

'Матрица корелляции Спирмена:'

,Пол,Возраст,Зарплата,Члены_семьи,Страховые_выплаты
Пол,1.000000,0.000134,0.011823,-0.006578,-0.000172
Возраст,0.000134,1.000000,-0.020238,-0.001503,0.548589
Зарплата,0.011823,-0.020238,1.000000,-0.025492,-0.005549
Члены_семьи,-0.006578,-0.001503,-0.025492,1.000000,-0.029210
Страховые_выплаты,-0.000172,0.548589,-0.005549,-0.029210,1.000000


'P-values:'

,Пол,Возраст,Зарплата,Члены_семьи,Страховые_выплаты
Пол,1.000000,0.992445,0.403252,0.641906,0.990285
Возраст,0.992445,1.000000,0.152472,0.915362,0.000000
Зарплата,0.403252,0.152472,1.000000,0.071488,0.694872
Члены_семьи,0.641906,0.915362,0.071488,1.000000,0.038884
Страховые_выплаты,0.990285,0.000000,0.694872,0.038884,1.000000


Значение корреляции Спирмена между `Cтраховыми выплатами` и `Возрастом` 0.548589 означает, что есть положительная связь между страховыми выплатами и возрастом. То есть, с увеличением возраста, вероятность страховых выплат также увеличивается. Допустим, что порог значимости p-value = 0.05, так как p-value в данном случае равен 0, можно сделать вывод, что эта связь не является случайной и статистически значима.

Значение корреляции Спирмена между `Cтраховыми выплатами` и количеством `Членов семьи` -0.029210 означает, что между переменными есть небольшая обратная связь. Так как p-value меньше уровня значимости 0.05, можно сделать вывод, что эта обратная связь не является случайной и статистически значима.

**Вывод** Обнаружена статистически значимая зависимость между `Возраст` и `Страховые_выплаты`, и `Члены_семьи` и `Страховые_выплаты`, что может говорить об уязвимости в конфиденциальности данных. Странностей в распределении данных не обнаружено, возможны единичные ошибки, которые не должны повлиять на датасет в целом

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Умножим изначальную матрицу признаков на обратимую матрицу $А$, получим измененную матрицу признаков:

$$
\tilde{X} = XA
$$

Тогда формула обучения с новыми признаками будет выглядеть так:

$$
\tilde{w} = (\tilde{X}^T \tilde{X})^{-1} \tilde{X}^T y
$$

Раскроем скобки:
$$
\tilde{w} = ((XA)^T XA)^{-1} (XA)^T y = (A^T X^T X A)^{-1}(A)^T (X)^T y 
$$

Неизвестно, является ли матрица $X$ обратимой, поэтому дальнейшее раскрытие скобок будет с учетом данного факта:

$$
\tilde{w} = A^{-1}(X^T X)^{-1} (A^T)^{-1} (A)^T (X)^T y
$$

Так как матрица $A$ по условию является обратимой, то $(A^T)^{-1} (A)^T=E$:

$$
\tilde{w} = A^{-1}(X^T X)^{-1} E (X)^T y = A^{-1}(X^T X)^{-1}(X)^T y = A^{-1}(X^T X)^{-1} X^T y
$$

Согласно формуле минимума MSE имеем $(X^T X)^{-1} X^T y = w$, следовательно:

$$
\tilde{w} = A^{-1}w
$$

Таким образом, параметры линейной регрессии в исходной задаче и в преобразованной связаны обратимой матрицей. При этом качество линейной регрессии не изменится, т.к. в пространстве признаков (векторов) был изменен только базис этих признаков.

**Ответ:** Качество линейной регрессии не поменяется.

**Обоснование:** Для исходных признаков имеются соответствующие веса и предсказания. Домножив исходные признаки на обратимую матрицу мы получили новые признаки, для которых были получены другие веса. Путем раскрытия скобок выше было установлено, что веса исходной матрицы и преобразованной связаны обратной матрицей, что в свою очередь означает, что предсказания для обоих случаеа - одинаковы:

$$
a = Xw
$$

$$
\tilde{a} = XA\tilde{w}
$$

$$
\tilde{w} = A^{-1}w
$$

$$
\tilde{a} = XAA^{-1}w = XEw = Xw = a
$$

Что и требовалось доказать.

## Алгоритм преобразования

**Алгоритм**

Домножение признаков на обратимую матрицу может помочь зашифровать данные клиентов страховой компании (например, путем устранения корелляции между признаками, которые могут раскрывать личную информацию о клиентах).

Этапы алгоритма линейного преобразования:
* Обучение модели линейной регресси на исходных данных, получение качества модели
* Генерация случайной матрицы, которая будет обратимой. Матрица должна иметь размерность $n * n$, где $n$ - размерность признаков
* Исследование зависимостей новых признаков
* Умножение признаков на обратимую матрицу
* Обучение модели линейной регрессии  на новых признаках
* Сравнение качества

**Обоснование**

Использование признаков с измененными зависимостями, по которым не возможно восстановить взаимосвязь, защитит конфоденциальность клиентов.

## Проверка алгоритма

*Определение фичей, таргетов и разделение выборок на тестовые и тренировочные:*

In [17]:
X = data.drop(['Страховые_выплаты'], axis=1)
y = data['Страховые_выплаты']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [19]:
model = LinearRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_train)
score_train = r2_score(y_train, predictions)
print('R2 Train:', score_train)

predictions = model.predict(X_test)
score_test = r2_score(y_test, predictions)
print('R2 Train:', score_test)

R2 Train: 0.42439431329760813
R2 Train: 0.4254778540696319


In [20]:
print("Число столбцов исходной матрицы:", data.shape[1])

Число столбцов исходной матрицы: 5


_Генерация случайной обратимой матрицы:_

In [21]:
def find_invertible_matrix(df):
    n = df.shape[1]
    while True:
        A = np.random.rand(n, n)
        inv_A = np.linalg.inv(A)
        if np.allclose(np.dot(A, inv_A), np.eye(n, dtype=int)):
            break
    return A

In [22]:
X1 = data.drop(['Страховые_выплаты'], axis=1)
y1 = data['Страховые_выплаты']

In [23]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.25, random_state=42)

In [24]:
print(X1_train.shape)
print(y1_train.shape)

(3750, 4)
(3750,)


In [25]:
invertable_matrix = find_invertible_matrix(X1_train)

*Умножение признаков на обратимую матрицу:*

In [26]:
print("Размерность сгенерированной обратимой матрицы train выборки:", invertable_matrix.shape)


Размерность сгенерированной обратимой матрицы train выборки: (4, 4)


In [27]:
df_dot_inv_train = np.dot(X1_train, invertable_matrix)
df_dot_inv_train = pd.DataFrame(df_dot_inv_train, columns=['Пол', 'Возраст', 'Зарплата', 'Члены_семьи'])

df_dot_inv_test = np.dot(X1_test, invertable_matrix)
df_dot_inv_test = pd.DataFrame(df_dot_inv_test, columns=['Пол', 'Возраст', 'Зарплата', 'Члены_семьи'])

Теперь данные запутаны и по ним нельзя восстановить зависимость.

In [28]:
model1 = LinearRegression()
model1.fit(df_dot_inv_train, y1_train)

predictions1 = model1.predict(df_dot_inv_train)
score1_train = r2_score(y1_train, predictions1)
print('R2 Train на исходных признаках:', score_train)
print('R2 Train на преобразованных признаках:', score1_train)
print()
predictions1 = model1.predict(df_dot_inv_test)
score1_test = r2_score(y1_test, predictions1)
print('R2 Test на исходных признаках:', score_test)
print('R2 Test на преобразованных признаках:', score1_test)

R2 Train на исходных признаках: 0.42439431329760813
R2 Train на преобразованных признаках: 0.4243943132976099

R2 Test на исходных признаках: 0.4254778540696319
R2 Test на преобразованных признаках: 0.4254778540696662


**Вывод:** В ходе проверки алгоритма было установлено, что качество моделей, обученных на исходных признаках, и качество модели на преобразованных существенно не изменилось.

Параметры линейной регрессии - это коэффициенты, которые определяют линейную зависимость между входными и выходными данными. В данном случае параметры простой линейной регрессии - это параметры веса (наклон) и сдвиг/байес (пересечение линии регрессии).

In [29]:
# получение наклона и пересечения линии регрессии для первой модели:
slope = model.coef_
print('Наклон:', slope)
intercept = model.intercept_
print('Пересечение:', intercept)

Наклон: [ 4.92432086e-03  3.51527196e-02 -2.45796619e-07 -1.49140089e-02]
Пересечение: -0.9200267704144679


In [30]:
# получение наклона и пересечения линии регрессии для второй модели:
slope1 = model1.coef_
print('Наклон:', slope1)
intercept1 = model1.intercept_
print('Пересечение:', intercept1)

Наклон: [-0.07677911 -0.02881579  0.02613351  0.07624925]
Пересечение: -0.9200267704144461


Веса и байес новой и старой модели существенно отличаются. Это может говорить о том, что преобразование данных существенно изменило распределение данных. Это может привести к тому, что модель, обученная на преобразованных данных, будет менее точной и надежной, чем модель, обученная на исходных данных.

## Выводы и рекомендации

Были загружены и предобработаны данные клиентов страховой компании. Проведено исследование зависимостей в данных, обнаружена корелляция между возрастом и страховыми выплатами клиентов, которые могут быть использованы злоумышленниками при попытках восстановления зависимостей данных. Был предложен алгоритм линейного преобразования данных с помощью домножения на обратимую матрицу, что позволизо существенно исказить зависимость наблюдаемых признаков без влияния на качество модели. Были обучены модели на исходных и преобразованных данных. Сравнение качества исследуемых моделей не показало существенный зезультат, что позволяет говорить о том, что алгоритм преобразования данных работает верно. Рекомендовано провести дополнительный анализ и оценить, насколько точно и надежно можно использовать модель, обученную на преобразованных данных.